# importamos librerias

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score as cv




# leemos datos

In [2]:

train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')


# vision general

In [3]:
train_data.sample(4)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
241,242,1,3,"Murphy, Miss. Katherine ""Kate""",female,NaN,1,0,367230,15.50,NaN,Q
191,192,0,2,"Carbines, Mr. William",male,19.00,0,0,28424,13.00,NaN,S
831,832,1,2,"Richards, Master. George Sibley",male,0.83,1,1,29106,18.75,NaN,S
365,366,0,3,"Adahl, Mr. Mauritz Nils Martin",male,30.00,0,0,C 7076,7.25,NaN,S


In [4]:
train_data.shape

(891, 12)

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
train_data.duplicated().sum()

0

In [7]:
train_data.isnull().sum().sort_values(ascending=False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

## cabin tiene demasiados por lo que la eliminaremos muy seguramente

In [8]:
train_data.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

## PassengerId es lógico ya que son ids y no aportan demasiado a la predicción junto a Name,Ticket y Fare.

In [9]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women) / len(women)
print("% de mujeres que sobrevivieron:", rate_women)

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men) / len(men)
print("% de hombres que sobrevivieron:", rate_men)

% de mujeres que sobrevivieron: 0.7420382165605095
% de hombres que sobrevivieron: 0.18890814558058924


## conclusiones

In [10]:
train_data = train_data.drop(columns=['Cabin','Fare','Ticket','Name','Age'])
test_data = test_data.drop(columns=['Cabin','Fare','Ticket','Name','Age'])


# preparación para modelo

In [11]:
X = train_data.drop(['Survived'],axis=1)
y = train_data['Survived']
features = ['Pclass','Sex','SibSp','Parch']

In [12]:
test_data.head()

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked
0,892,3,male,0,0,Q
1,893,3,female,1,0,S
2,894,2,male,0,0,Q
3,895,3,male,0,0,S
4,896,3,female,1,1,S


## pasamos columnas categóricas a numéricas

In [13]:
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])


# definimos modelo

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd

# Parámetros a probar
n_estimators = [10, 50, 100, 200]
max_depth = [2, 3, 5, 7,10]

best_score = 0
best_params = {}

# Bucle para encontrar los mejores parámetros
for n in n_estimators:
    for d in max_depth:
        model = RandomForestClassifier(n_estimators=n, max_depth=d, random_state=32)
        score = cross_val_score(model, X, y, cv=5).mean()
        if score > best_score:
            best_score = score
            best_params = {'n_estimators': n, 'max_depth': d}

# Entrenar con los mejores parámetros
best_model = RandomForestClassifier(**best_params, random_state=32)
best_model.fit(X, y)
predictions = best_model.predict(X_test)

# Crear CSV de envío
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)

print("Mejores parámetros:", best_params)
print("Mejor accuracy:", best_score)
print("CSV de envío generado.")


Mejores parámetros: {'n_estimators': 100, 'max_depth': 3}
Mejor accuracy: 0.8080723118448307
CSV de envío generado.
